In [1]:
from box import ConfigBox
import os

In [2]:
exaple = ConfigBox({
    "key": "value",
    "key2": 123,
    "key3": [1, 2, 3],
    "key4": (1, 2, 3),})

In [3]:
exaple.key2

123

In [4]:
from ensure import ensure_annotations
@ensure_annotations
def get_prod(x:int ,y:int)->int:
    return x * y
    

In [5]:
get_prod(2, 3)

6

In [6]:
get_prod(2, "3")

EnsureError: Argument y of type <class 'str'> to <function get_prod at 0x0000021A9A235BD0> does not match annotation type <class 'int'>

In [7]:
%pwd

'd:\\BISM\\datascience_mlops\\research'

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [9]:
os.chdir("../")

In [10]:
%pwd


'd:\\BISM\\datascience_mlops'

In [11]:
from src.data_science_project.utils.common import read_yaml, create_directories
from src.data_science_project.constants import *
from src.data_science_project.constants import PARAMS_FILE_PATH


In [12]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config= self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir)
        return data_ingestion_config

    

In [13]:
import os
import urllib.request as requests    
from src.data_science_project import logger
import zipfile

In [18]:
class Dataingestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    #download the file from the source URL and save it to local path

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            file_name,header = requests.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{file_name} downloaded successfully")
        else:
            logger.info(f"File already exists at {self.config.local_data_file}")
        


    def extract_zip_file(self):
        unzib_path=self.config.unzip_dir
        os.makedirs(unzib_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzib_path)
        


In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    dataingestion = Dataingestion(config=data_ingestion_config)
    dataingestion.download_file()
    dataingestion.extract_zip_file()
except Exception as e:
    raise e 

[2025-05-02 15:19:02,467: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-02 15:19:02,470: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-02 15:19:02,475: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-02 15:19:02,479: INFO: common: created directory at: artifacts]
[2025-05-02 15:19:02,484: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-02 15:19:02,487: INFO: 2525656938: File already exists at artifacts/data_ingestion/data.zip]
